# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
%run create_tables.py

In [3]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [4]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files('data/song_data')

In [6]:
filepath = song_files[0]
print(filepath)

/workspace/home/data/song_data/A/B/B/TRABBNP128F932546F.json


In [7]:
import json
with open(filepath) as f:
    song_data = json.load(f)
df = pd.read_json(filepath, lines=True)
df.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,AR62SOJ1187FB47BB5,NaN,,NaN,Chase & Status,337.68444,1,SOGVQGJ12AB017F169,Ten Tonne,2005


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
songs_columns = ['song_id', 'title', 'artist_id','year', 'duration']
song_data = df[songs_columns].values.tolist()

In [9]:
print(song_data)

[['SOGVQGJ12AB017F169', 'Ten Tonne', 'AR62SOJ1187FB47BB5', 2005, 337.68444]]


In [10]:
song_data= song_data[0]
song_data

['SOGVQGJ12AB017F169', 'Ten Tonne', 'AR62SOJ1187FB47BB5', 2005, 337.68444]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [12]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist()

In [13]:
artist_data = artist_data[0]
artist_data

['AR62SOJ1187FB47BB5', 'Chase & Status', '', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
# obtain list of all log JSON files in data/log data
log_files = get_files("data/log_data/")
log_files[0]

'/workspace/home/data/log_data/2018/11/2018-11-04-events.json'

In [16]:
# Get first log file in list
filepath = log_files[0]

In [17]:
# read log file and view data
df = pd.read_json(filepath, lines=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Theodore,M,0,Smith,NaN,free,"Houston-The Woodlands-Sugar Land, TX",GET,Home,1.540306e+12,154,None,200,1541290555796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,52
1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",PUT,NextSong,1.540896e+12,124,Mean Ol'World,200,1541292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99
2,None,Logged In,Jahiem,M,0,Miles,NaN,free,"San Antonio-New Braunfels, TX",GET,Home,1.540817e+12,42,None,200,1541299033796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,En Mi Mundo,200,1541300092796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43
4,Lifehouse,Logged In,Jahiem,M,2,Miles,203.59791,free,"San Antonio-New Braunfels, TX",PUT,NextSong,1.540817e+12,42,We'll Never Know,200,1541300337796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
# Filter data by NextSong
time_data_df = df[df['page'] == 'NextSong'].reset_index()

In [19]:
# Convert ts timestamp column to datetime
time_data_df['ts'] = pd.to_datetime(time_data_df['ts'])
time_data_df['ts'].head()

0   1970-01-01 00:25:41.292603796
1   1970-01-01 00:25:41.300092796
2   1970-01-01 00:25:41.300337796
3   1970-01-01 00:25:41.300540796
4   1970-01-01 00:25:41.306152796
Name: ts, dtype: datetime64[ns]

In [20]:
# Extract the timestamp, hour, day, week of year, month, year, and weekday from the ts column 
#set time_data to a list containing these values in order

time_data_df['hour'] = time_data_df['ts'].dt.hour.astype(int)
time_data_df['day'] = time_data_df['ts'].dt.day.astype(int)
time_data_df['week'] = time_data_df['ts'].dt.week.astype(int)
time_data_df['month'] = time_data_df['ts'].dt.month.astype(int)
time_data_df['year'] = time_data_df['ts'].dt.year.astype(int)
time_data_df['weekday'] = time_data_df['ts'].dt.dayofweek.astype(int)
time_data_df.head()

,index,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,...,status,ts,userAgent,userId,hour,day,week,month,year,weekday
0,1,Professor Longhair,Logged In,Ann,F,0,Banks,214.20363,free,"Salt Lake City, UT",...,200,1970-01-01 00:25:41.292603796,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; r...,99,0,1,1,1,1970,3
1,3,Gary Hobbs,Logged In,Jahiem,M,1,Miles,245.52444,free,"San Antonio-New Braunfels, TX",...,200,1970-01-01 00:25:41.300092796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43,0,1,1,1,1970,3
2,4,Lifehouse,Logged In,Jahiem,M,2,Miles,203.59791,free,"San Antonio-New Braunfels, TX",...,200,1970-01-01 00:25:41.300337796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43,0,1,1,1,1970,3
3,5,Olivia Ruiz,Logged In,Jahiem,M,3,Miles,254.74567,free,"San Antonio-New Braunfels, TX",...,200,1970-01-01 00:25:41.300540796,"""Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537....",43,0,1,1,1,1970,3
4,7,Jordan Rudess,Logged In,Cecilia,F,1,Owens,1367.84934,free,"Atlanta-Sandy Springs-Roswell, GA",...,200,1970-01-01 00:25:41.306152796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:32.0) G...,6,0,1,1,1,1970,3


In [21]:
time_data_df = time_data_df[['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']] 
time_data = time_data_df.values.tolist()
print(time_data[0: 5])


[[Timestamp('1970-01-01 00:25:41.292603796'), 0, 1, 1, 1, 1970, 3], [Timestamp('1970-01-01 00:25:41.300092796'), 0, 1, 1, 1, 1970, 3], [Timestamp('1970-01-01 00:25:41.300337796'), 0, 1, 1, 1, 1970, 3], [Timestamp('1970-01-01 00:25:41.300540796'), 0, 1, 1, 1, 1970, 3], [Timestamp('1970-01-01 00:25:41.306152796'), 0, 1, 1, 1, 1970, 3]]


In [22]:
# create column labels
column_labels = column_labels = ['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']
print(column_labels)

['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']


In [23]:
# tie data to columns
time_dict = {c:[] for c in column_labels}

# use loop to combine column_labels and time_data values into the dictionary
for row in time_data[1:]:
    for c,v in zip(column_labels, row):
        time_dict[c].append(v)      

time_df = pd.DataFrame.from_dict(time_dict)
time_df.head()

,ts,hour,day,week,month,year,weekday
0,1970-01-01 00:25:41.300092796,0,1,1,1,1970,3
1,1970-01-01 00:25:41.300337796,0,1,1,1,1970,3
2,1970-01-01 00:25:41.300540796,0,1,1,1,1970,3
3,1970-01-01 00:25:41.306152796,0,1,1,1,1970,3
4,1970-01-01 00:25:41.310546796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [24]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [25]:
# Set user ID, first name, last name, gender and level and set to user_df
user_df = df[['userId','firstName','lastName','gender','level']]
user_df = user_df.drop_duplicates()
user_df = user_df[user_df["userId"]!=""]
user_df.head()

,userId,firstName,lastName,gender,level
0,52,Theodore,Smith,M,free
1,99,Ann,Banks,F,free
2,43,Jahiem,Miles,M,free
6,6,Cecilia,Owens,F,free
8,25,Jayden,Graves,M,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [26]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [27]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
         
    if results:
        songid = results[0]
        artistid = results[1]
    else:
        songid, artistid = None, None

    #insert songplay record
               
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)

    
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [29]:
conn.close()

In [30]:
%run create_tables.py

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

In [31]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
import subprocess 


def process_song_file(cur, filepath):
    # open song file
    df = pd.read_json(filepath, lines=True)

    # insert song record
    song_data = df[['song_id','title','artist_id','year','duration']].values.tolist()
    song_data = list(song_data[0])
    cur.execute(song_table_insert, song_data)
    
    # insert artist record
    artist_data = artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']].values.tolist() 
    artist_data = list(artist_data[0])
    cur.execute(artist_table_insert, artist_data)


def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextSong action
    df = df[df['page'] == 'NextSong'].reset_index()

    # convert timestamp column to datetime
    t = df['ts'] = pd.to_datetime(df['ts'], unit='ms')
    df['hour'] = df['ts'].dt.hour.astype(int)
    df['day'] = df['ts'].dt.day.astype(int)
    df['week'] = df['ts'].dt.week.astype(int)
    df['month'] = df['ts'].dt.month.astype(int)
    df['year'] = df['ts'].dt.year.astype(int)
    df['weekday'] = df['ts'].dt.dayofweek.astype(int)
    
       
    # tie data to columns
    time_dict = {c:[] for c in column_labels}

    # use loop to combine column_labels and time_data values into the dictionary
    for row in time_data[1:]:
        for c,v in zip(column_labels, row):
            time_dict[c].append(v)      

    
    # insert time data records
    time_data = df[['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']].values.tolist()
    column_labels = ['ts', 'hour', 'day', 'week', 'month', 'year', 'weekday']
    time_df = pd.DataFrame.from_dict(time_dict)

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df[['userId','firstName','lastName','gender','level']]
    user_df = user_df.drop_duplicates()
    user_df = user_df[user_df["userId"]!=""]

    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, row)

    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        songplay_data = songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)


def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))


def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()

    subprocess.run(['python', 'create_tables.py'])  # Call create_tables.py script
    
    process_data(cur, conn, filepath='data/song_data', func=process_song_file)
    process_data(cur, conn, filepath='data/log_data', func=process_log_file)

    conn.close()


if __name__ == "__main__":
    main()

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file

IntegrityError: duplicate key value violates unique constraint "artists_pkey"
DETAIL:  Key (artist_id)=(ARNTLGG11E2835DDB9) already exists.
